# Example 3 - Realtime Synchronization With OSC Timestamps

The third example demonstrates how we can use OSC timestamps to improve synchronization and real-time musical performance over a network. The code is similar to the metronome control system introduced in example 2 but adds the option to adjust the timestamps associated with each OSC packet.

The OSC timestamps are essentially NTP timestamps; a 64-bit fixed floating point number. The first 32 bits specify the number of seconds since midnight on January 1, 1900, and the last 32 bits specify fractional parts of a second to a precision of about 200 picoseconds. Fortunately, to customize the timestamp in python-osc, we only have to provide a floating point number of seconds since the epoch (midnight on January 1, 1900) in UTC, python-osc will do the rest for us. Similarly, in PD, the complexity is hidden as we just need to specify timestampOffset messages to the [mrpeach/packOSC] object to manipulate the timestamp of the next incoming OSC message.


### Files 
1. Notebook "Example 3 - Realtime Synchronization With OSC Timestamps"
2. Metronome tick "./tick.wav"
3. Pure Data patch example1.pd from "../pd/example3.pd"
4. Metronome tick "../pd/tick.wav"
5. Notebook "Example 4 - UTC Timestamp Printing"

<p align="left">
 <img src="../../fig/example3.jpg" width=500>
</p>

In [ ]:
# setup our connection variables 
# remote client ip and port
clientIp = '127.0.0.1'
clientPort = 8001

# local server ip and port
serverIp = '127.0.0.1'
serverPort = 8000

## Connect with PD client

In [ ]:
from pythonosc import udp_client

# Setup a client connection and send messages to PD
client = udp_client.UDPClient(clientIp, clientPort)
print(f'Starting client on {clientIp}, port {clientPort}.')

In [ ]:
from pythonosc import osc_bundle_builder
from pythonosc import osc_message_builder
from datetime import datetime
import calendar
import time

# select how many seconds to offset OSC timestamp before playback
timestampOffset = 1

# convert current DateTime to UTC UNIX Timestamp (seconds since epoch).
date = datetime.utcnow()
utc_timestamp = calendar.timegm(date.utctimetuple())

# send OSC frames (custom bundles) with custom timestamp to the client
while True:
    # open a OSC bundle
    # add a custom timestamp to the bundle builder object, instead of osc_bundle_builder.IMMEDIATELY.
    bundle = osc_bundle_builder.OscBundleBuilder(utc_timestamp+timestampOffset)

    # create a message with an OSC address
    msg = osc_message_builder.OscMessageBuilder(address="/pd")

    # add arguments to the message
    msg.add_arg("tick")

    # build the message and add to current bundle
    bundle.add_content(msg.build())

    # close OSC bundle
    bundle = bundle.build()

    # send the bundle to remote client
    client.send(bundle)

    # One frame every half second
    time.sleep(.5)

print("done sending...")

## Serve an OSC server

When receiving an OSC bundle with a timestamp, python-osc will automatically evalute it and either delay, play or discard the incoming frame.

In [ ]:
from pythonosc import dispatcher
from pythonosc import osc_server
from playsound import playsound

audio = "./tick.wav"

# Setup different "routes" where we can map different functions to different OSC adressess we receive.
def pyHandler(address, args):
    print(f'{address} {args}')
    
    if args == "tick":
        # play the tick audio file from the root dir
        playsound(audio)

dispatcher = dispatcher.Dispatcher()
dispatcher.map("/py*", pyHandler)

# A simple OSC threading server to listen for OSC messages
server = osc_server.ThreadingOSCUDPServer((serverIp, serverPort), dispatcher)
print(f'Starting server on {server.server_address[0]}, port {server.server_address[1]}.')

# run our server
server.serve_forever()

In [ ]:
# closing the OSC server
server.server_close()

## Activity

1. Test and explore the example on your local machine. Use "localhost" or "127.0.0.1" as your client and server IPs.
2. By offsetting/forwarding the OSC timestamp, you should experience that your signal is more stable and resilient to jitter. According to Schmeder et.al, we can use something called Forward Synchronization to remove jitter and better synchronize two machines if the clock sync error is smaller than the average transport jitter. Try to recreate the Forward Synchronization method detailed in the literature and diagram below. Also, make the clock signal control something more interesting than a simple metronome.

<p align="left">
 <img src="../../fig/forward-sync.jpg" width=500>
</p>